In [3]:
pip install firebase-admin

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
print(sys.version)


3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]


In [5]:
pip install transformers torch emoji

Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests

API_KEY = '58b4bcc7e574420ca04d721e21e233cd'
url = ('https://newsapi.org/v2/top-headlines?'
       'language=en&'
       'pageSize=100&'
       'apiKey=' + API_KEY)

response = requests.get(url)
articles = response.json().get('articles', [])

In [7]:
news_data = []
for article in articles:
    news_data.append({
        'title': article.get('title', ''),
        'description': article.get('description', ''),
        'content': article.get('content', ''),
        'publishedAt': article.get('publishedAt', ''),
        'source': article.get('source', {}).get('name', ''),  # safely get nested 'name'
        'author': article.get('author', ''),
        'url': article.get('url', ''),
        'urlToImage': article.get('urlToImage', '')
    })

print(f"Fetched {len(news_data)} articles")


Fetched 27 articles


In [8]:
import json

# Print the first article in a nicely formatted way
print(json.dumps(news_data[0], indent=4))

{
    "title": "How Did 10 Inmates Escape a New Orleans Jail Without Anyone Noticing? - WSJ",
    "description": null,
    "content": null,
    "publishedAt": "2025-05-24T09:30:00Z",
    "source": "The Wall Street Journal",
    "author": "WSJ",
    "url": "https://www.wsj.com/us-news/new-orleans-jail-break-security-4f81f8da",
    "urlToImage": null
}


In [9]:
   pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [10]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [12]:
def process_news_data(news_data):
    df = pd.DataFrame(news_data)
    df_clean = df.dropna(subset=['content'])
    
    # Process text: tokenize, remove stopwords, lemmatize
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
        
    def clean_text(text):
        if not isinstance(text, str):
            return ""
        tokens = word_tokenize(text.lower())
        filtered = [word for word in tokens if word.isalnum() and word not in stop_words]
        return " ".join(lemmatizer.lemmatize(word) for word in filtered)
    df_clean.loc[:, 'processed_content'] = df_clean['content'].apply(clean_text)

    
    return df_clean

In [13]:
processed_df = process_news_data(news_data)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15712\4179575933.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:, 'processed_content'] = df_clean['content'].apply(clean_text)


In [14]:
print(f"Processed {len(processed_df)} of {len(news_data)} news items")
print(processed_df[['title', 'processed_content']])

Processed 26 of 27 news items
                                                title  \
1   Russia launches major aerial attack on Ukraine...   
2   Knicks-Pacers: 5 takeaways as Pascal Siakam po...   
3   Horoscope for Saturday, May 24, 2025 - Chicago...   
4   ​New COVID variant NB.1.8.1 behind surge in ca...   
5   White House dismisses scores of National Secur...   
6   San Diego plane crash is a devastating loss to...   
7   Eric Adams appears to be running a real campai...   
8   See a penny, pick it up? In the future, probab...   
9   German woman arrested after mass stabbing at H...   
10  Immigrant arrests at courthouses signal new ta...   
11  Billy Joel cancels shows due to a brain disord...   
12  4 top partners quit Paul Weiss, Big Law firm t...   
13  Trump announces U.S. Steel, Nippon ‘partnershi...   
14  ‘Lilo & Stitch’ Targeting $170M-$180M Box Offi...   
15  Emotions were on full display during final epi...   
16  For U.S. Jews, D.C. museum killings deepen res...   
1

In [15]:
from transformers import pipeline
import pandas as pd

# Assuming you already have 'processed_df' from your previous code
sentiment_analyzer = pipeline("sentiment-analysis")

# Analyze sentiment
results = sentiment_analyzer(processed_df['processed_content'].tolist())

# Add sentiment to DataFrame
processed_df["sentiment"] = [res["label"] for res in results]
processed_df["sentiment_score"] = [res["score"] for res in results]


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [16]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("newssen-1ba2b-firebase-adminsdk-fbsvc-16eb43bcaa.json")
firebase_admin.initialize_app(cred)



In [17]:
db = firestore.client()
for index, row in processed_df.iterrows():
    data = {
        'title': row['title'],
        'content': row['processed_content'],
        'sentiment': row['sentiment'],
        'score': row['sentiment_score']
    }
    db.collection('news_articles').add(data)

print("Articles uploaded successfully!")

Articles uploaded successfully!
